In [ ]:
import requests
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()


In [ ]:
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/Dodstal'

data = {
  "query": [
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}


In [ ]:
r = requests.post(url,json=data)
print (r.status_code)

json = r.json()

In [ ]:
json['data'][0]

In [ ]:
#parse death data from json to df 

data_list = []

nr_records = (len(json['data']))

for rec in range(nr_records):

    data_list.append ((json['data'][rec]['key'][0],json['data'][rec]['key'][1],
                       json['data'][rec]['key'][2],json['data'][rec]['values'][0]))

In [ ]:
# death rate df
death_rate = pd.DataFrame(data_list,columns=['age_grp','gender','year','dead_per_1k'])
death_rate

In [ ]:
death_rate['dead_per_1k'] = death_rate['dead_per_1k'].astype(float)

In [ ]:
gender_groups  = death_rate.groupby('gender')

In [ ]:
male_death_rate = gender_groups.get_group('1').copy()
male_death_rate['age_grp'] = pd.Categorical(male_death_rate['age_grp'],
                                            ['0','1-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39',
                                            '40-44','45-49','50-54','55-59','60-64','65-69','70-74',
                                            '75-79','80-84','85-89','90+'])

male_death_rate = pd.pivot(male_death_rate,index='age_grp',columns='year',values='dead_per_1k')
male_death_rate

In [ ]:
female_death_rate = gender_groups.get_group('2').copy()
female_death_rate['age_grp'] = pd.Categorical(female_death_rate['age_grp'],
                                            ['0','1-4','5-9','10-14','15-19','20-24','25-29','30-34','35-39',
                                            '40-44','45-49','50-54','55-59','60-64','65-69','70-74',
                                            '75-79','80-84','85-89','90+'])

female_death_rate = pd.pivot(female_death_rate,index='age_grp',columns='year',values='dead_per_1k')
female_death_rate

In [ ]:
ax = male_death_rate.plot(figsize=(18,12),color='b',style='x--',legend=False,
                         title='Sweden deaths per 1K per age group, men (blue) vs women (red),2000-2019')
female_death_rate.plot(ax=ax,color='r',style='o--',legend=False)

plt.yscale('log')
plt.ylabel('deaths per 1K [log scale]')

_= plt.xticks(range(len(male_death_rate)),male_death_rate.index.values)

plt.savefig('deaths_men_darwin_award.jpg',format='jpg',dpi=1200)